In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'gabor'
CHANNEL = 'green'
PARAM_CSV = "gabor_new.csv"

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_gabor_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,47.17,0.00,-0.00,0.0,0.00,-0.00,-0.00,-0.00,0.0,-0.00,...,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00
1,0.00,19.01,0.00,-0.0,-0.00,-0.00,-0.00,0.00,-0.0,-0.00,...,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00
2,-0.00,0.00,7.43,-0.0,0.00,0.00,0.00,-0.00,0.0,-0.00,...,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00
3,0.00,-0.00,-0.00,2.9,-0.00,0.00,-0.00,-0.00,-0.0,0.00,...,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
4,0.00,-0.00,0.00,-0.0,1.07,-0.00,0.00,-0.00,-0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
5,-0.00,-0.00,0.00,0.0,-0.00,0.31,0.00,-0.00,0.0,-0.00,...,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00
6,-0.00,-0.00,0.00,-0.0,0.00,0.00,0.18,0.00,0.0,0.00,...,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00
7,-0.00,0.00,-0.00,-0.0,-0.00,-0.00,0.00,61.51,0.0,0.00,...,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
8,0.00,-0.00,0.00,-0.0,-0.00,0.00,0.00,0.00,26.7,0.00,...,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00004,-0.00001,0.00004,0.00001,-0.00004,-0.00004,-0.00004,0.00002,-0.00002,...,-0.00002,0.00007,-0.00001,0.00003,0.00000,-0.00003,0.00002,-0.00000,-0.00000,0.00001
1,0.00004,1.00000,0.00005,-0.00006,-0.00000,-0.00004,-0.00001,0.00005,-0.00001,-0.00003,...,-0.00000,0.00000,-0.00000,0.00003,-0.00001,-0.00000,-0.00003,0.00002,0.00002,0.00004
2,-0.00001,0.00005,1.00000,-0.00003,0.00000,0.00003,0.00003,-0.00000,0.00004,-0.00001,...,0.00001,-0.00002,0.00003,-0.00004,0.00004,-0.00000,-0.00008,-0.00001,0.00004,-0.00001
3,0.00004,-0.00006,-0.00003,1.00000,-0.00002,0.00003,-0.00003,-0.00005,-0.00003,0.00000,...,-0.00002,-0.00005,0.00002,0.00002,-0.00003,0.00003,0.00002,0.00003,-0.00004,-0.00000
4,0.00001,-0.00000,0.00000,-0.00002,1.00000,-0.00002,0.00001,-0.00000,-0.00000,0.00007,...,0.00000,0.00002,0.00001,0.00002,0.00003,-0.00004,0.00004,0.00002,0.00005,-0.00004
5,-0.00004,-0.00004,0.00003,0.00003,-0.00002,1.00000,0.00001,-0.00005,0.00003,-0.00001,...,-0.00003,-0.00002,0.00004,-0.00004,0.00000,0.00001,0.00003,0.00001,-0.00000,-0.00001
6,-0.00004,-0.00001,0.00003,-0.00003,0.00001,0.00001,1.00000,0.00002,0.00000,0.00005,...,0.00003,-0.00001,-0.00002,-0.00002,-0.00001,0.00001,-0.00003,0.00004,-0.00003,0.00001
7,-0.00004,0.00005,-0.00000,-0.00005,-0.00000,-0.00005,0.00002,1.00000,0.00000,0.00003,...,-0.00001,0.00003,-0.00003,0.00001,-0.00004,-0.00001,-0.00004,0.00004,0.00003,-0.00004
8,0.00002,-0.00001,0.00004,-0.00003,-0.00000,0.00003,0.00000,0.00000,1.00000,0.00003,...,-0.00002,0.00001,0.00000,0.00003,0.00003,-0.00006,0.00002,0.00004,-0.00002,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.015208483344210443

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.04999997e+01 6.78368022e+01 5.66935535e+01 5.45790927e+01
 5.21574399e+01 4.70022312e+01 1.69543452e+01 1.09803304e+01
 9.48766067e+00 8.62765711e+00 8.01614762e+00 7.28811294e+00
 2.47776820e+00 1.53896235e+00 1.33218687e+00 1.27113887e+00
 1.18377668e+00 1.04505110e+00 4.03871367e-01 2.37388889e-01
 2.03448889e-01 1.91785353e-01 1.72624669e-01 1.46962400e-01
 5.44182432e-02 3.33522554e-02 2.82982084e-02 2.63262309e-02
 2.35622654e-02 1.85267199e-02 5.95452492e-03 3.50571690e-03
 2.73310789e-03 2.33264586e-03 2.14684430e-03 1.49569763e-03
 7.71942295e-04 5.40992604e-04 2.43093330e-04 1.84739314e-04
 1.46423055e-04 1.37490900e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.037410,-0.006628,-0.002385,-0.000930,-0.000340,-0.000103,-0.000054,0.994497,-0.010454,-0.003315,...,-0.000309,-0.000090,-0.000025,-0.041778,-0.007094,-0.002358,-0.000920,-0.000337,-0.000106,-0.000027
1,-0.079853,-0.007340,-0.002559,-0.000996,-0.000364,-0.000103,-0.000061,0.057097,-0.012090,-0.003574,...,-0.000325,-0.000094,-0.000025,-0.105091,-0.007820,-0.002494,-0.000965,-0.000371,-0.000117,-0.000025
2,-0.224119,-0.004576,-0.001569,-0.000596,-0.000218,-0.000067,-0.000036,0.019949,-0.007818,-0.002190,...,-0.000194,-0.000058,-0.000015,0.924023,-0.004921,-0.001527,-0.000602,-0.000215,-0.000070,-0.000018
3,-0.656448,-0.001830,-0.000610,-0.000231,-0.000085,-0.000020,-0.000014,0.007002,-0.003158,-0.000834,...,-0.000076,-0.000028,-0.000007,0.076211,-0.001900,-0.000593,-0.000240,-0.000087,-0.000029,-0.000007
4,0.636378,-0.012452,-0.004188,-0.001582,-0.000583,-0.000171,-0.000101,0.043817,-0.021848,-0.005878,...,-0.000532,-0.000153,-0.000043,0.267110,-0.013355,-0.004131,-0.001581,-0.000593,-0.000190,-0.000045
5,0.323278,-0.024735,-0.008097,-0.003073,-0.001134,-0.000332,-0.000184,0.067529,-0.044874,-0.011445,...,-0.001037,-0.000310,-0.000078,0.234030,-0.026516,-0.007967,-0.003056,-0.001143,-0.000363,-0.000084
6,0.020208,-0.037384,-0.007589,-0.002780,-0.001014,-0.000292,-0.000165,0.013069,0.994205,-0.011336,...,-0.000926,-0.000274,-0.000071,0.019176,-0.043675,-0.007545,-0.002737,-0.001015,-0.000326,-0.000077
7,0.011109,-0.090238,-0.007895,-0.002741,-0.001020,-0.000293,-0.000163,0.007706,0.042781,-0.012406,...,-0.000924,-0.000265,-0.000072,0.010724,-0.155892,-0.007720,-0.002721,-0.001011,-0.000316,-0.000077
8,0.009415,-0.252042,-0.008123,-0.002797,-0.001009,-0.000294,-0.000165,0.006683,0.030132,-0.013201,...,-0.000922,-0.000267,-0.000074,0.009068,0.947533,-0.008049,-0.002756,-0.001030,-0.000322,-0.000077
9,0.009039,0.905032,-0.008924,-0.003066,-0.001102,-0.000326,-0.000180,0.006449,0.026898,-0.014882,...,-0.001003,-0.000297,-0.000078,0.008736,0.167566,-0.008819,-0.003019,-0.001108,-0.000348,-0.000080


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.005502784349421419,
 0.015792670131354436,
 0.07597736102191133,
 0.2507774169449859,
 0.3636224715861984,
 0.14843173932172216,
 0.005794826939795827,
 0.021880671512889527,
 0.05246737842788329,
 0.09496809524257421,
 0.23988820550153633,
 0.27624244011061283,
 0.0074414446033037596,
 0.02890124984286979,
 0.16069264110321335,
 0.23782237978786935,
 0.2809236652723702,
 0.3557777349303439,
 0.01060619750321723,
 0.035528577511372506,
 0.2120363715850272,
 0.29946664895261244,
 0.2602158419676427,
 0.30168049253288975,
 0.017224771123751648,
 0.04777847981134997,
 0.2394745840662924,
 0.35219993355260604,
 0.29409211887765796,
 0.38265871060267487,
 0.026400175078753363,
 0.05652761494750369,
 0.10362028253993771,
 0.23723982919532483,
 0.24330263381478412,
 0.42005081461388416,
 0.25359267827715204,
 0.34452205754527976,
 0.16500771128124125,
 0.28817688604299807,
 0.22954262625588495,
 0.4830075561259969]